In [50]:
import numpy as np
import scipy.stats as stats
m = 1000
n = 10
d = 2
z_distr = stats.multivariate_normal(mean = np.ones(d), cov = np.eye(d))
epsilon_distr = stats.multivariate_normal(mean = np.ones(n), cov = np.eye(n))


In [51]:
Z = z_distr.rvs(size = m)

In [52]:
Z.shape

(1000, 2)

In [53]:
W = np.random.uniform(low=0.0, high=1.0, size=(n, d))

In [54]:
W.shape

(10, 2)

In [55]:
epsilon = epsilon_distr.rvs(size = m)

In [56]:
nu, std = np.zeros(n), 1
X = Z@W.T
X = Z@W.T + epsilon + nu


In [57]:
def expectation_step(W, nu, std, X, n, m, d):
    std_z = np.linalg.inv(np.eye(d)  + (std**(-2)) * W.T@W)
    mean_z = (X - np.repeat(nu.reshape(1, -1), m, axis = 0)) @ ((std**(-2)) * std_z @ W.T).T
    return (mean_z, std_z)

mean_z, std_z = expectation_step(W, nu, std, X, n, m, d)

In [58]:
print(Z)

[[ 0.32826469  1.98425404]
 [-0.47733538  0.62172797]
 [-1.36634315 -0.18233339]
 ...
 [ 1.99367444  0.12527334]
 [ 0.60758008 -1.17460774]
 [ 1.29731588  0.42038265]]


In [59]:
print((mean_z))

[[ 1.17646072  2.35430395]
 [ 0.47404612  1.11615182]
 [-0.46043904  1.0319853 ]
 ...
 [ 2.74092086  1.14986699]
 [ 1.03781264  0.27258335]
 [ 1.08658219  1.11539964]]


In [60]:
print(W)

[[0.01452794 0.16652798]
 [0.3969708  0.34774918]
 [0.54744719 0.30597731]
 [0.89950752 0.68598575]
 [0.88240152 0.53831544]
 [0.7154917  0.8355805 ]
 [0.96745316 0.12768992]
 [0.91564348 0.13277442]
 [0.54289209 0.95508827]
 [0.17619995 0.88819796]]


In [61]:
def maximization_step(mean_z, std_z, nu, std, X, n, m, d):
    mean_mean_z = np.mean(mean_z, axis = 0)
    mean_X = np.mean(X, axis = 0)
    X0 = X - np.repeat(mean_X.reshape(1, -1), m, axis = 0)
    mean_z0 = mean_z - np.repeat(mean_mean_z.reshape(1, -1), m, axis = 0)
    W =  X0.T @ (np.linalg.inv((mean_z0).T@(mean_z0) - std_z) @ (mean_z0).T).T
    nu = mean_X - W @ mean_mean_z
    std = np.sqrt(np.mean(W@ std_z @ W.T))
    return W, nu, std


In [62]:
W_, nu_, std_ = W, nu, std
for i in range(100):
    mean_z, std_z = expectation_step(W_, nu_, std_, X, n, m, d)
    W_, nu_, std_ = maximization_step(mean_z, std_z, nu_, std_, X, n, m, d)
    if (np.linalg.norm(Z@W.T - X) / (n * m) < 0.01 *std):
        break

In [63]:
print((W_, nu_, std_))

(array([[-0.08833821,  0.37270874],
       [ 0.49953254,  0.32565996],
       [ 0.62625636,  0.37942504],
       [ 1.13844196,  0.69642404],
       [ 1.05173884,  0.53354028],
       [ 0.63553497,  1.11587018],
       [ 1.37813623, -0.12082277],
       [ 1.35612563, -0.21219083],
       [ 0.40242829,  1.43789174],
       [-0.02937775,  1.33248278]]), array([ 0.80716924,  0.49858763,  0.24783375, -0.28282494, -0.0649696 ,
       -0.10346159, -0.00436239,  0.1567013 , -0.23526414,  0.1540653 ]), 2.07236463732001e-53)


In [64]:
print(np.linalg.norm(Z@W_.T - X) / (n * m))

0.013424642649548195


In [65]:
print((W, nu, std))

(array([[0.01452794, 0.16652798],
       [0.3969708 , 0.34774918],
       [0.54744719, 0.30597731],
       [0.89950752, 0.68598575],
       [0.88240152, 0.53831544],
       [0.7154917 , 0.8355805 ],
       [0.96745316, 0.12768992],
       [0.91564348, 0.13277442],
       [0.54289209, 0.95508827],
       [0.17619995, 0.88819796]]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 1)


In [66]:
epsilon = epsilon_distr.rvs(size = m)
Z_test = z_distr.rvs(size = m)
X_test = Z_test@W.T + epsilon + nu

In [76]:
from sklearn.decomposition import PCA
pca = PCA(n_components=d)
pca.fit(X)
Z_pca = pca.fit_transform(X_test)
Z_bayes = X_test@ (np.linalg.inv(W_.T@W_) @ W.T).T
X_pedict = pca.inverse_transform(Z_pca)
X_pedict_bayes = Z_bayes@W_.T
print("Незашумленные данные")
print(np.linalg.norm(X_pedict - Z_test@W.T) / (n * m))
print(np.linalg.norm(X_pedict_bayes - Z_test@W.T) / (n * m))
print("Шумные данные")
print(np.linalg.norm(X_pedict - X_test) / (n * m))
print(np.linalg.norm(X_pedict_bayes - X_test) / (n * m))


Незашумленные данные
0.011044672599926903
0.007606374029180619
Шумные данные
0.009013487088044558
0.010405886844817597


Байес дает прогнозы ближе к незашумленным данным, PCA приближает к шумным данным 

## Часть 2